In [112]:
import pandas as pd
import random
from datetime import datetime, timedelta

In [113]:
# CRIA FUNÇÕES DE ACESSO AO BANCO DE DADOS
import psycopg2

# cria conexão
def conn():
  con = psycopg2.connect(
    database = "drogasystem", 
    user = "oscar", 
    password = "oscar", 
    host = "127.0.0.1", 
    port = "5432")
  cur = con.cursor()
  return con, cur

# faz selects
def select(sql):
  all = []
  con, cur = conn()
  cur.execute(sql)
  all = cur.fetchall()
  cur.close()
  con.close()
  return all

# executa inserts e updates
def exec(sql):
  try:
    con, cur = conn()
    cur.execute(sql)
    con.commit()
  except (Exception, psycopg2.Error) as error:
    if cur == None: cur.rollback()
    print("Error:", error)
  finally:
    if cur: cur.close()
    if con: con.close()

In [114]:
# CRIA DATAFRAME DA TABELA DE MEDICAMENTO
cols = ["ID", "LABORATORIO_ID", "NOME", "SUBSTANCIA", "DOSAGEM", "APRESENTACAO", 
        "TIPO", "CONTROLE", "QUANTIDADE", "GGREM", "CLASSE", "PRECO"]
sql = f"select {','.join(cols)} from medicamento order by 1;"
df_medicamento = pd.DataFrame(select(sql), columns=cols)
df_medicamento.head()

,ID,LABORATORIO_ID,NOME,SUBSTANCIA,DOSAGEM,APRESENTACAO,TIPO,CONTROLE,QUANTIDADE,GGREM,CLASSE,PRECO
0,1,1,AMPICILINA,AMPICILINA,50 MG/ML,Líquido,Genérico,AM,150 ML,507701907133117,PENICILINAS ORAIS DE AMPLO ESPECTRO,74.11
1,2,2,NESINA MET,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA,"(12,5 + 1000,0) MG",Sólido,Novo,Liberado,60,501115110025705,ASSOCIAÇÕES DE INIBIDORES DPP-IV COM BIGUANIDAS,303.77
2,3,3,TEGRETOL,CARBAMAZEPINA,200 MG,Sólido,Novo,C1,60,526515003110311,ANTIEPILÉPTICOS,89.22
3,4,4,TRELIM,CLORIDRATO DE SERTRALINA,100 MG,Sólido,Similar,C1,60,528519120163307,ANTIDEPRESSIVOS SSRI,401.24
4,5,5,ZETERINA,EZETIMIBA,10 MG,Sólido,Similar,Liberado,60,543817070024603,TODOS OUTROS REGULADORES DE COLESTEROL E TRIGL...,277.35


In [115]:
# CRIA DATAFRAME DO EXCEL
df_excel = pd.read_excel("cmed_final_v3.xlsx"
          ).filter(items=[4, 11, "Probabilidade", "Validade"]
          ).rename(columns={4: "GGREM", 11: "CUSTO", "Probabilidade": "PROBABILIDADE", "Validade": "VALIDADE"})
df_excel.head()

,GGREM,CUSTO,PROBABILIDADE,VALIDADE
0,507701907133117,53.61,2.5,180
1,501115110025705,228.00,2.5,180
2,526515003110311,64.54,2.5,180
3,528519120163307,290.24,2.5,180
4,543817070024603,200.62,2.5,180


In [116]:
# CRIA DATAFRAME DA TABELA DE CLIENTE
df_cliente = pd.DataFrame(select("select id from cliente;"), columns=["ID"])
df_cliente.head()

,ID
0,1
1,2
2,3
3,4
4,5


In [117]:
# CRIA DATAFRAME DA TABELA DE PROFISSIONAL
df_profissional = pd.DataFrame(select("select id from profissional;"), columns=["ID"])
df_profissional.head()

,ID
0,21
1,22
2,23
3,24
4,25


In [118]:
# CRIA FUNÇÃO INSERE ESTOQUE
def insert_estoque(i, data):
  # acessa medicamento_id
  medicamento_id = int(df_medicamento.loc[i, "ID"])

  # cria lote
  lote = f"lote {random.randrange(0, 999999):06d}"

  # procura linha do excel com base no ggrem
  excel = df_excel[df_excel["GGREM"] == df_medicamento.loc[i, "GGREM"]].reset_index(drop=True)

  # acessa custo
  custo = float(excel["CUSTO"][0])

  # calcula data da validade
  validade = int(excel["VALIDADE"][0])
  validade = data + timedelta(days = validade)
  
  # calcula quant_inicial
  probabilidade = float(excel["PROBABILIDADE"][0])
  # quant_inicial = random.randrange(10, quantidade, 10)
  quant_inicial = int(probabilidade / 10 + 1) * 10

  # gera insert do estoque
  sql = "insert into estoque values "
  sql += f"(default, {medicamento_id}, '{lote}', '{data}', '{validade}', {custo}, {quant_inicial}, 0, {quant_inicial}); \n"

  # gera insert ou update do estoque_total
  if select(f"select count(*) from estoque_total where medicamento_id = {medicamento_id};")[0][0] == 0:
    minimo = 5 if probabilidade < 50 else 10
    sql += f"insert into estoque_total values (default, {medicamento_id}, {quant_inicial}, {minimo}, 9999); "
  else:
    sql += f"update estoque_total set total = total + {quant_inicial} where medicamento_id = {medicamento_id}; "
    
  return sql + "\n"

In [119]:
# CRIA DATAFRAME DA TABELA DE ESTOQUE
excel = []
for i in range(1, df_excel.shape[0]):
  excel.append(f"select {df_excel.loc[i, 'GGREM']} ggrem, {df_excel.loc[i, 'PROBABILIDADE']} probabilidade")
excel = ' union '.join(excel)

cols = ["ESTOQUE_ID", "MEDICAMENTO_ID", "QUANT_ATUAL", "GGREM", 
        "PRECO", "CONTROLE", "PROBABILIDADE"]

# produtos em estoque
sql_excel  = "select estoque.id, estoque.medicamento_id, estoque.quant_atual, "
sql_excel += "medicamento.ggrem, medicamento.preco, medicamento.controle, "
sql_excel += "excel.probabilidade "
sql_excel += "from estoque "
sql_excel += "inner join medicamento on medicamento.id = estoque.medicamento_id "
sql_excel += f"inner join ({excel}) excel on excel.ggrem = medicamento.ggrem "

sql_excel += "where quant_atual > 0 "
sql_excel += "and estoque.id in (select min(id) from estoque "
sql_excel += "where quant_atual > 0 group by medicamento_id) "
sql_excel += "order by 1;"

def tabela_estoque():
  global df_estoque
  df_estoque = pd.DataFrame(select(sql_excel), columns=cols)
  df_estoque["PROBABILIDADE"] = df_estoque["PROBABILIDADE"].astype(float)


In [120]:
# CRIA FUNÇÃO INSERE VENDA
def insert_venda(data_hora, venda_id):
  sql = ""
  totais = 0

  # cliente e profissional
  cliente_id = "null"
  profissional_id = "null"
  if random.randrange(10) > 5:
    cliente_id = df_cliente.loc[random.choice(df_cliente.index), 'ID']

  # atualiza df estoque a cada venda
  tabela_estoque()

  quantidade_itens_venda = random.randrange(3) + 1
  for q in range(quantidade_itens_venda):
    
    # escolha do item
    i = random.choices(df_estoque.index, weights=df_estoque["PROBABILIDADE"])[0]

    estoque_id = int(df_estoque.loc[i, "ESTOQUE_ID"])
    medicamento_id = int(df_estoque.loc[i, "MEDICAMENTO_ID"])
    quant_atual = int(df_estoque.loc[i, "QUANT_ATUAL"])
    preco = float(df_estoque.loc[i, "PRECO"])

    # verifica profissional
    controle = str(df_estoque.loc[i, "CONTROLE"])
    if controle != "Liberado": 
      profissional_id = df_profissional.loc[random.choice(df_profissional.index), 'ID']

    # calcula quantidade
    if preco > 100:
      quantidade = 1
    elif preco > 30:
      quantidade = random.randrange(2) + 1
    else:
      quantidade = random.randrange(3) + 1
    if quantidade > quant_atual: quantidade = quant_atual

    # calcula desconto
    desconto = 0
    if cliente_id != "null":
      desconto = quantidade * preco * random.randrange(10) / 100
    elif random.randrange(10) > 0:
      desconto = quantidade * preco * random.randrange(5) / 100
    desconto = int(desconto * 10) / 10
    
    # totais
    total = quantidade * preco - desconto
    totais += total

    # cria insert e update do item de venda
    sql += "insert into itens_venda values "
    sql += f"(default, {venda_id}, {medicamento_id}, {estoque_id}, {quantidade}, {desconto:.2f}, {total:.2f}); \n"
    sql += f"update estoque set quant_venda = quant_venda + {quantidade}, quant_atual = quant_atual - {quantidade} where id = {estoque_id}; \n"
    sql += f"update estoque_total set total = total - {quantidade} where medicamento_id = {medicamento_id}; \n"

    # trata estoque baixo
    estoque_total = select(f"select total, minimo from estoque_total where medicamento_id = {medicamento_id};")[0]
    if estoque_total[0] < estoque_total[1]:
      # procura índice do medicamento
      j = df_medicamento[df_medicamento["GGREM"] == df_estoque.loc[i, "GGREM"]].index[0]
      sql += insert_estoque(j, data_hora.date())

  sql = f"insert into venda values ({venda_id}, '{data_hora}', {totais:.2f}, {cliente_id}, {profissional_id}); \n" + sql
  print(sql)
  exec(sql)


In [121]:
# GERA INSERTS DO ESTOQUE INICIAL
if select("select count(*) from estoque;")[0][0] == 0:
  sql = ""
  data = datetime.today() - timedelta(days = 366)
  data = data.date()
  for i in df_medicamento.index: sql += insert_estoque(i, data)
  print(sql)
  exec(sql)

insert into estoque values (default, 1, 'lote 481763', '2022-11-02', '2023-05-01', 53.61, 10, 0, 10); 
insert into estoque_total values (default, 1, 10, 5, 9999); 
insert into estoque values (default, 2, 'lote 674269', '2022-11-02', '2023-05-01', 228.0, 10, 0, 10); 
insert into estoque_total values (default, 2, 10, 5, 9999); 
insert into estoque values (default, 3, 'lote 141171', '2022-11-02', '2023-05-01', 64.54, 10, 0, 10); 
insert into estoque_total values (default, 3, 10, 5, 9999); 
insert into estoque values (default, 4, 'lote 948775', '2022-11-02', '2023-05-01', 290.24, 10, 0, 10); 
insert into estoque_total values (default, 4, 10, 5, 9999); 
insert into estoque values (default, 5, 'lote 374442', '2022-11-02', '2023-05-01', 200.62, 10, 0, 10); 
insert into estoque_total values (default, 5, 10, 5, 9999); 
insert into estoque values (default, 6, 'lote 329376', '2022-11-02', '2023-05-01', 199.18, 10, 0, 10); 
insert into estoque_total values (default, 6, 10, 5, 9999); 
insert into e

In [122]:
# data inicial das vendas
data = select("select coalesce(max(time_stamp), current_date - 366) from venda;")[0][0]

# venda_id inicial
venda_id = select("select coalesce(max(id), 0) as id from venda;")[0][0]

sql = ""

# período de venda
for i in range(365):
  data += timedelta(days = 1)

  # começa as vendas pela manhã
  data_hora = data + timedelta(hours = 6)

  # quantidade de vendas no dia
  qtd_vnd_dia = random.randrange(14)
  f = 10000 if qtd_vnd_dia < 8 else 5000
  for j in range(qtd_vnd_dia):
    venda_id += 1

    # intervalo entre cada venda
    data_hora += timedelta(seconds = random.randrange(f))

    insert_venda(data_hora, venda_id)


insert into venda values (1, '2022-11-03 06:02:32', 43.40, null, null); 
insert into itens_venda values (default, 1, 376, 376, 3, 1.30, 43.40); 
update estoque set quant_venda = quant_venda + 3, quant_atual = quant_atual - 3 where id = 376; 
update estoque_total set total = total - 3 where medicamento_id = 376; 

insert into venda values (2, '2022-11-03 08:12:56', 192.26, 4, null); 
insert into itens_venda values (default, 2, 255, 255, 1, 3.50, 47.22); 
update estoque set quant_venda = quant_venda + 1, quant_atual = quant_atual - 1 where id = 255; 
update estoque_total set total = total - 1 where medicamento_id = 255; 
insert into itens_venda values (default, 2, 418, 418, 2, 0.50, 54.80); 
update estoque set quant_venda = quant_venda + 2, quant_atual = quant_atual - 2 where id = 418; 
update estoque_total set total = total - 2 where medicamento_id = 418; 
insert into itens_venda values (default, 2, 127, 127, 2, 0.90, 90.24); 
update estoque set quant_venda = quant_venda + 2, quant_atua